## Inicializar la Dimensión Cliente
Importaciones

In [100]:
import pandas as pd
from sqlalchemy import create_engine
import yaml
from datetime import  timedelta

# Añadir conexión a la base de datos

In [101]:
with open('../config.yml', 'r') as f:
    config = yaml.safe_load(f)
    config_ryf = config['RAPIDOS-Y_FURIOSOS']
    config_etl = config['ETL_RYF']

# Construir la URL de conexión para cada base de datos
url_ryf = (f"{config_ryf['drivername']}://{config_ryf['user']}:{config_ryf['password']}@"
           f"{config_ryf['host']}:{config_ryf['port']}/{config_ryf['dbname']}")

url_etl = (f"{config_etl['drivername']}://{config_etl['user']}:{config_etl['password']}@"
           f"{config_etl['host']}:{config_etl['port']}/{config_etl['dbname']}")

# Crear el Engine de SQLAlchemy para ambas conexiones
ryf_conn = create_engine(url_ryf)
etl_conn = create_engine(url_etl)

# Consulta para extraer la información de servicios con sus novedades

In [102]:
query = """
SELECT 
    ns.*,
    ms.id as servicio_mensajeria_id,
    ms.descripcion,
    ms.nombre_solicitante,
    ms.fecha_solicitud,
    ms.hora_solicitud,
    ms.fecha_deseada,
    ms.hora_deseada,
    ms.nombre_recibe,
    ms.telefono_recibe,
    ms.descripcion_pago,
    ms.ida_y_regreso,
    ms.activo,
    ms.novedades,
    ms.cliente_id,
    ms.mensajero_id as mensajero_id_ms,  -- Renombrar esta columna
    ms.tipo_pago_id,
    ms.tipo_servicio_id,
    ms.tipo_vehiculo_id,
    ms.usuario_id,
    ms.prioridad,
    ms.ciudad_destino_id,
    ms.ciudad_origen_id,
    ms.hora_visto_por_mensajero,
    ms.visto_por_mensajero,
    ms.descripcion_multiples_origenes,
    ms.mensajero2_id,
    ms.mensajero3_id,
    ms.multiples_origenes,
    ms.asignar_mensajero,
    ms.es_prueba,
    ms.descripcion_cancelado,
    cu.sede_id
FROM
    public.mensajeria_novedadesservicio ns
LEFT JOIN
    public.mensajeria_servicio ms 
    ON ns.servicio_id = ms.id 
LEFT JOIN
    public.clientes_usuarioaquitoy cu 
    ON ms.usuario_id = cu.id
ORDER BY ns.id ASC;
"""

df_datamart = pd.read_sql(query, ryf_conn)
df_datamart.head()


,id,fecha_novedad,tipo_novedad_id,descripcion,servicio_id,es_prueba,mensajero_id,servicio_mensajeria_id,descripcion,nombre_solicitante,...,hora_visto_por_mensajero,visto_por_mensajero,descripcion_multiples_origenes,mensajero2_id,mensajero3_id,multiples_origenes,asignar_mensajero,es_prueba,descripcion_cancelado,sede_id
0,4,2023-11-30 05:00:00+00:00,1,A,51,True,7,51,Recoger muestras 8:20 am,chat_GPT,...,None,,,7.0,7.0,False,False,True,,7
1,5,2023-11-30 05:00:00+00:00,1,Halo,51,True,7,51,Recoger muestras 8:20 am,chat_GPT,...,None,,,7.0,7.0,False,False,True,,7
2,6,2023-11-30 05:00:00+00:00,1,A,51,True,7,51,Recoger muestras 8:20 am,chat_GPT,...,None,,,7.0,7.0,False,False,True,,7
3,7,2023-11-30 05:00:00+00:00,1,B,51,True,7,51,Recoger muestras 8:20 am,chat_GPT,...,None,,,7.0,7.0,False,False,True,,7
4,8,2023-11-30 05:00:00+00:00,1,A,51,True,7,51,Recoger muestras 8:20 am,chat_GPT,...,None,,,7.0,7.0,False,False,True,,7


# LLamar las dimensiones para poder hacer la transformación

In [103]:
df_dim_cliente = pd.read_sql('SELECT * FROM dim_cliente', etl_conn)
df_dim_mensajero = pd.read_sql('SELECT * FROM dim_mensajero', etl_conn)
df_dim_sede = pd.read_sql('SELECT * FROM dim_sede', etl_conn)
df_dim_hora = pd.read_sql('SELECT * FROM dim_hora', etl_conn)
df_dim_fecha = pd.read_sql('SELECT * FROM dim_fecha', etl_conn)

df_dim_fecha.head()


,key_dim_fecha,fecha_completa,fecha_id,año,mes,nombre_mes,semana,dia,nombre_dia,trimestre,is_Holiday,holiday,saved,weekend,es_dia_laboral,es_fin_de_semana,es_feriado
0,0,2023-01-01,20230101,2023,1,January,52,1,Sunday,1,True,Año Nuevo,2024-11-04,True,False,True,True
1,1,2023-01-02,20230102,2023,1,January,1,2,Monday,1,False,None,2024-11-04,False,True,False,False
2,2,2023-01-03,20230103,2023,1,January,1,3,Tuesday,1,False,None,2024-11-04,False,True,False,False
3,3,2023-01-04,20230104,2023,1,January,1,4,Wednesday,1,False,None,2024-11-04,False,True,False,False
4,4,2023-01-05,20230105,2023,1,January,1,5,Thursday,1,False,None,2024-11-04,False,True,False,False


# cambiar cliente info por dim cliente

In [104]:
# Merge para obtener key_dim_cliente basado en cliente_id
df_datamart = df_datamart.merge(df_dim_cliente[['cliente_id', 'key_dim_cliente']],
                                on='cliente_id',
                                how='left')
# Eliminar la columna cliente_id original
df_datamart.drop(columns=['cliente_id'], inplace=True)

# Renombrar la columna a cliente_id si es necesario
df_datamart.rename(columns={'key_dim_cliente': 'cliente_id'}, inplace=True)

df_datamart.head()

,id,fecha_novedad,tipo_novedad_id,descripcion,servicio_id,es_prueba,mensajero_id,servicio_mensajeria_id,descripcion,nombre_solicitante,...,visto_por_mensajero,descripcion_multiples_origenes,mensajero2_id,mensajero3_id,multiples_origenes,asignar_mensajero,es_prueba,descripcion_cancelado,sede_id,cliente_id
0,4,2023-11-30 05:00:00+00:00,1,A,51,True,7,51,Recoger muestras 8:20 am,chat_GPT,...,,,7.0,7.0,False,False,True,,7,7
1,5,2023-11-30 05:00:00+00:00,1,Halo,51,True,7,51,Recoger muestras 8:20 am,chat_GPT,...,,,7.0,7.0,False,False,True,,7,7
2,6,2023-11-30 05:00:00+00:00,1,A,51,True,7,51,Recoger muestras 8:20 am,chat_GPT,...,,,7.0,7.0,False,False,True,,7,7
3,7,2023-11-30 05:00:00+00:00,1,B,51,True,7,51,Recoger muestras 8:20 am,chat_GPT,...,,,7.0,7.0,False,False,True,,7,7
4,8,2023-11-30 05:00:00+00:00,1,A,51,True,7,51,Recoger muestras 8:20 am,chat_GPT,...,,,7.0,7.0,False,False,True,,7,7


# cambiar sede info por dim sede

In [105]:
df_dim_sede.head()

,key_dim_sede,sede_id,sede_nombre,sede_direccion,sede_telefono,contacto_sede,ciudad_nombre,cliente_nombre
0,0,10,FARALLONES /123,Los angeles distrito Latino,310-70000,JUAN PEREZ,CALI,CRUZ AZUL-LIFE
1,1,11,REMEDIOZ/ 123,Los angeles distrito Latino,310-70000,JUAN PEREZ,CALI,CRUZ AZUL-LIFE
2,2,13,DIME / LOS ROJOS,Los angeles distrito Latino,310-70000,JUAN PEREZ,CALI,CRUZ AZUL-LIFE
3,3,14,DESPACHOS / LOS ROJOS,Los angeles distrito Latino,310-70000,JUAN PEREZ,CALI,CRUZ AZUL-LIFE
4,4,23,POPAYAN BODEGA 28 / A,Los angeles distrito Latino,310-70000,JUAN PEREZ,POPAYAN,CARROS DEL PACIFICO (CHINA)


In [106]:
# Merge para obtener key_dim_sede basado en sede_id
df_datamart = df_datamart.merge(df_dim_sede[['sede_id', 'key_dim_sede']],
                                on='sede_id',
                                how='left')
# Eliminar la columna sede_id original
df_datamart.drop(columns=['sede_id'], inplace=True)

# Renombrar la columna a sede_id si es necesario
#df_datamart.rename(columns={'key_dim_sede': 'sede_id'}, inplace=True)

df_datamart.head()

,id,fecha_novedad,tipo_novedad_id,descripcion,servicio_id,es_prueba,mensajero_id,servicio_mensajeria_id,descripcion,nombre_solicitante,...,visto_por_mensajero,descripcion_multiples_origenes,mensajero2_id,mensajero3_id,multiples_origenes,asignar_mensajero,es_prueba,descripcion_cancelado,cliente_id,key_dim_sede
0,4,2023-11-30 05:00:00+00:00,1,A,51,True,7,51,Recoger muestras 8:20 am,chat_GPT,...,,,7.0,7.0,False,False,True,,7,17
1,5,2023-11-30 05:00:00+00:00,1,Halo,51,True,7,51,Recoger muestras 8:20 am,chat_GPT,...,,,7.0,7.0,False,False,True,,7,17
2,6,2023-11-30 05:00:00+00:00,1,A,51,True,7,51,Recoger muestras 8:20 am,chat_GPT,...,,,7.0,7.0,False,False,True,,7,17
3,7,2023-11-30 05:00:00+00:00,1,B,51,True,7,51,Recoger muestras 8:20 am,chat_GPT,...,,,7.0,7.0,False,False,True,,7,17
4,8,2023-11-30 05:00:00+00:00,1,A,51,True,7,51,Recoger muestras 8:20 am,chat_GPT,...,,,7.0,7.0,False,False,True,,7,17


In [107]:
# Merge para obtener key_dim_cliente basado en cliente_id
df_datamart = df_datamart.merge(df_dim_cliente[['cliente_id', 'key_dim_cliente']],
                                on='cliente_id',
                                how='left')
# Convertir key_dim_cliente a entero inmediatamente después del merge
df_datamart['key_dim_cliente'] = df_datamart['key_dim_cliente'].astype(int)
# Eliminar la columna cliente_id original
df_datamart.drop(columns=['cliente_id'], inplace=True)



In [108]:
df_datamart.head()

,id,fecha_novedad,tipo_novedad_id,descripcion,servicio_id,es_prueba,mensajero_id,servicio_mensajeria_id,descripcion,nombre_solicitante,...,visto_por_mensajero,descripcion_multiples_origenes,mensajero2_id,mensajero3_id,multiples_origenes,asignar_mensajero,es_prueba,descripcion_cancelado,key_dim_sede,key_dim_cliente
0,4,2023-11-30 05:00:00+00:00,1,A,51,True,7,51,Recoger muestras 8:20 am,chat_GPT,...,,,7.0,7.0,False,False,True,,17,8
1,5,2023-11-30 05:00:00+00:00,1,Halo,51,True,7,51,Recoger muestras 8:20 am,chat_GPT,...,,,7.0,7.0,False,False,True,,17,8
2,6,2023-11-30 05:00:00+00:00,1,A,51,True,7,51,Recoger muestras 8:20 am,chat_GPT,...,,,7.0,7.0,False,False,True,,17,8
3,7,2023-11-30 05:00:00+00:00,1,B,51,True,7,51,Recoger muestras 8:20 am,chat_GPT,...,,,7.0,7.0,False,False,True,,17,8
4,8,2023-11-30 05:00:00+00:00,1,A,51,True,7,51,Recoger muestras 8:20 am,chat_GPT,...,,,7.0,7.0,False,False,True,,17,8


# cambiar mensajero info por dim mensajero

In [109]:
# Merge para obtener key_dim_mensajero basado en mensajero_id
df_datamart = df_datamart.merge(df_dim_mensajero[['mensajero_id', 'key_dim_mensajero']],
                                on='mensajero_id',
                                how='left')

# Convertir key_dim_mensajero a entero inmediatamente después del merge
df_datamart['key_dim_mensajero'] = df_datamart['key_dim_mensajero'].astype(int)

# Eliminar la columna mensajero_id original
df_datamart.drop(columns=['mensajero_id'], inplace=True)



In [110]:
df_datamart.head()


,id,fecha_novedad,tipo_novedad_id,descripcion,servicio_id,es_prueba,servicio_mensajeria_id,descripcion,nombre_solicitante,fecha_solicitud,...,descripcion_multiples_origenes,mensajero2_id,mensajero3_id,multiples_origenes,asignar_mensajero,es_prueba,descripcion_cancelado,key_dim_sede,key_dim_cliente,key_dim_mensajero
0,4,2023-11-30 05:00:00+00:00,1,A,51,True,51,Recoger muestras 8:20 am,chat_GPT,2023-11-14,...,,7.0,7.0,False,False,True,,17,8,37
1,5,2023-11-30 05:00:00+00:00,1,Halo,51,True,51,Recoger muestras 8:20 am,chat_GPT,2023-11-14,...,,7.0,7.0,False,False,True,,17,8,37
2,6,2023-11-30 05:00:00+00:00,1,A,51,True,51,Recoger muestras 8:20 am,chat_GPT,2023-11-14,...,,7.0,7.0,False,False,True,,17,8,37
3,7,2023-11-30 05:00:00+00:00,1,B,51,True,51,Recoger muestras 8:20 am,chat_GPT,2023-11-14,...,,7.0,7.0,False,False,True,,17,8,37
4,8,2023-11-30 05:00:00+00:00,1,A,51,True,51,Recoger muestras 8:20 am,chat_GPT,2023-11-14,...,,7.0,7.0,False,False,True,,17,8,37


# cambiar fecha info por dim fecha
## Fecha solicitud y fecha completa

In [111]:
# Convertir fecha_solicitud y fecha_completa a formato datetime64[ns]
df_datamart['fecha_solicitud'] = pd.to_datetime(df_datamart['fecha_solicitud'])
df_dim_fecha['fecha_completa'] = pd.to_datetime(df_dim_fecha['fecha_completa'])

# Realizar el merge para obtener key_dim_fecha correspondiente a cada fecha_solicitud
df_datamart = df_datamart.merge(df_dim_fecha[['fecha_completa', 'key_dim_fecha']],
                                left_on='fecha_solicitud',
                                right_on='fecha_completa',
                                how='left')

# Renombrar la columna key_dim_fecha a key_dim_fecha_solicitud y eliminar la columna fecha_completa
df_datamart.rename(columns={'key_dim_fecha': 'key_dim_fecha_solicitud'}, inplace=True)
df_datamart.drop(columns=['fecha_completa', 'fecha_solicitud'], inplace=True)

df_datamart.head()

,id,fecha_novedad,tipo_novedad_id,descripcion,servicio_id,es_prueba,servicio_mensajeria_id,descripcion,nombre_solicitante,hora_solicitud,...,mensajero2_id,mensajero3_id,multiples_origenes,asignar_mensajero,es_prueba,descripcion_cancelado,key_dim_sede,key_dim_cliente,key_dim_mensajero,key_dim_fecha_solicitud
0,4,2023-11-30 05:00:00+00:00,1,A,51,True,51,Recoger muestras 8:20 am,chat_GPT,15:00:56,...,7.0,7.0,False,False,True,,17,8,37,317
1,5,2023-11-30 05:00:00+00:00,1,Halo,51,True,51,Recoger muestras 8:20 am,chat_GPT,15:00:56,...,7.0,7.0,False,False,True,,17,8,37,317
2,6,2023-11-30 05:00:00+00:00,1,A,51,True,51,Recoger muestras 8:20 am,chat_GPT,15:00:56,...,7.0,7.0,False,False,True,,17,8,37,317
3,7,2023-11-30 05:00:00+00:00,1,B,51,True,51,Recoger muestras 8:20 am,chat_GPT,15:00:56,...,7.0,7.0,False,False,True,,17,8,37,317
4,8,2023-11-30 05:00:00+00:00,1,A,51,True,51,Recoger muestras 8:20 am,chat_GPT,15:00:56,...,7.0,7.0,False,False,True,,17,8,37,317


## Fecha deseada

In [112]:
# Convertir fecha_deseada y fecha_completa a formato datetime64[ns] si no se ha hecho ya
df_datamart['fecha_deseada'] = pd.to_datetime(df_datamart['fecha_deseada'])
df_dim_fecha['fecha_completa'] = pd.to_datetime(df_dim_fecha['fecha_completa'])

# Realizar el merge para obtener key_dim_fecha correspondiente a cada fecha_deseada
df_datamart = df_datamart.merge(df_dim_fecha[['fecha_completa', 'key_dim_fecha']],
                                left_on='fecha_deseada',
                                right_on='fecha_completa',
                                how='left')

# Renombrar la columna key_dim_fecha a key_dim_fecha_deseada y eliminar la columna fecha_completa
df_datamart.rename(columns={'key_dim_fecha': 'key_dim_fecha_deseada'}, inplace=True)
df_datamart.drop(columns=['fecha_completa', 'fecha_deseada'], inplace=True)



# fecha novedad

In [113]:
# Convertir fecha_novedad a formato datetime64[ns] y extraer solo la fecha
df_datamart['fecha_novedad'] = pd.to_datetime(df_datamart['fecha_novedad']).dt.normalize()  # Esto ajusta solo a la fecha

# Extraer solo la hora en una nueva columna
df_datamart['hora_novedad'] = pd.to_datetime(df_datamart['fecha_novedad']).dt.time

# Realizar el merge para obtener key_dim_fecha correspondiente a cada fecha_novedad
df_datamart = df_datamart.merge(df_dim_fecha[['fecha_completa', 'key_dim_fecha']],
                                left_on='fecha_novedad',
                                right_on='fecha_completa',
                                how='left')

# Renombrar la columna key_dim_fecha a key_dim_fecha_novedad y eliminar la columna fecha_completa
df_datamart.rename(columns={'key_dim_fecha': 'key_dim_fecha_novedad'}, inplace=True)
df_datamart.drop(columns=['fecha_completa'], inplace=True)

# Verificar el resultado
df_datamart.head()


ValueError: You are trying to merge on datetime64[ns, UTC] and datetime64[ns] columns for key 'fecha_novedad'. If you wish to proceed you should use pd.concat

In [48]:
df_datamart.head()

,id,fecha_novedad,tipo_novedad_id,descripcion,servicio_id,es_prueba,servicio_mensajeria_id,descripcion,nombre_solicitante,fecha_solicitud,...,descripcion_multiples_origenes,mensajero2_id,mensajero3_id,multiples_origenes,asignar_mensajero,es_prueba,descripcion_cancelado,key_dim_sede,key_dim_cliente,key_dim_mensajero
0,4,2023-11-30 05:00:00+00:00,1,A,51,True,51,Recoger muestras 8:20 am,chat_GPT,2023-11-14,...,,7.0,7.0,False,False,True,,17,8,37
1,5,2023-11-30 05:00:00+00:00,1,Halo,51,True,51,Recoger muestras 8:20 am,chat_GPT,2023-11-14,...,,7.0,7.0,False,False,True,,17,8,37
2,6,2023-11-30 05:00:00+00:00,1,A,51,True,51,Recoger muestras 8:20 am,chat_GPT,2023-11-14,...,,7.0,7.0,False,False,True,,17,8,37
3,7,2023-11-30 05:00:00+00:00,1,B,51,True,51,Recoger muestras 8:20 am,chat_GPT,2023-11-14,...,,7.0,7.0,False,False,True,,17,8,37
4,8,2023-11-30 05:00:00+00:00,1,A,51,True,51,Recoger muestras 8:20 am,chat_GPT,2023-11-14,...,,7.0,7.0,False,False,True,,17,8,37
